# Final Project

In [6]:
import warnings
warnings.filterwarnings('ignore')

from helper import get_train_test, lowercase, tokenize, remove_punctuation, remove_stopwords, \
    remove_non_letters, stemming, correct_spelling, reduce_lengthening, subsampling, save_preprocessed_data, \
    get_categories, get_train_test_data, get_label_encoder
    
config = {
    'lowercase': True,
    'stemming': True,
    'remove_stopwords': True,
    'remove_non_letters': True,
    'remove_punctuation': True,
    'correct_spelling': True,
    'reduce_lengthening': True,
    'subsampling': False,
}

## Preprocessing

In [1]:
train, test = get_train_test_data('train_data.txt', 'test_data.txt')

# subsampling
if config['subsampling']:
    subsampling(train)
    subsampling(test)

# lowercasing
if config['lowercase']:
    lowercase(train)
    lowercase(test)

# removing non letters
if config['remove_non_letters']:
    remove_non_letters(train)
    remove_non_letters(test)

# removing punctuation
if config['remove_punctuation']:
    remove_punctuation(train)
    remove_punctuation(test)

# tokenizing
tokenize(train)
tokenize(test)

# reducing length
if config['reduce_lengthening']:
    reduce_lengthening(train)
    reduce_lengthening(test)
    
# correcting spelling
if config['correct_spelling']:
    correct_spelling(train)
    correct_spelling(test)

# removing stopwords
if config['remove_stopwords']:
    remove_stopwords(train)
    remove_stopwords(test)
    
# stemming
if config['stemming']:
    stemming(train)
    stemming(test)
    
# saving preprocessed data
save_preprocessed_data(train, 'train_data.pkl')
save_preprocessed_data(test, 'test_data.pkl')

## Checkpoint

In [2]:
from helper import load_preprocessed_data

train, test = load_preprocessed_data('train_data.pkl', 'test_data.pkl')

## Classifications

### Nearest Centroid

In [5]:
# TODO

### LSTM (Deep Learning)

In [4]:
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.preprocessing import text, sequence
from numpy import argmax

hyper_params = {
    'validation_split': 0.1,
    'batch_size': 32,
    'epochs': 10,
    'embedding_size': 128,
    'keep_probability': 0.9,
    'lstm_size': 50,
    'dense_size': 50,
    'max_sequence': 100
}

categories = get_categories('categories.txt')
label_encoder = get_label_encoder(categories)
X_train, y_train, X_test = get_train_test(train, test, label_encoder)
vocabulary = set(X_train).union(set(X_test))
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(vocabulary)
train_tokenized = tokenizer.texts_to_sequences(X_train)
test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(train_tokenized, maxlen=hyper_params['max_sequence'])
X_test = sequence.pad_sequences(test_tokenized, maxlen=hyper_params['max_sequence'])

model = Sequential()
model.add(Embedding(len(vocabulary) + 1, hyper_params['embedding_size']))
model.add(LSTM(hyper_params['lstm_size']))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(hyper_params['dense_size'], activation='relu'))
model.add(Dropout(1 - hyper_params['keep_probability']))
model.add(Dense(len(categories), activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

history = model.fit(X_train, y_train, batch_size=hyper_params['batch_size'], epochs=hyper_params['epochs'], 
                    validation_split=hyper_params['validation_split'])

predictions = [label_encoder.inverse_transform([argmax(result)])[0] for result in model.predict(X_test[-5000:])]
output_file = open('output2.txt', 'w')
for item in predictions:
    output_file.write("%s\n" % item)

Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 67s 4ms/step - loss: 0.2033 - acc: 0.9473 - val_loss: 0.1714 - val_acc: 0.9545
Epoch 2/10
18000/18000 [==============================] - 76s 4ms/step - loss: 0.1627 - acc: 0.9550 - val_loss: 0.1512 - val_acc: 0.9566
Epoch 3/10
18000/18000 [==============================] - 68s 4ms/step - loss: 0.1315 - acc: 0.9604 - val_loss: 0.1289 - val_acc: 0.9612
Epoch 4/10
18000/18000 [==============================] - 76s 4ms/step - loss: 0.0976 - acc: 0.9682 - val_loss: 0.1180 - val_acc: 0.9646
Epoch 5/10
18000/18000 [==============================] - 70s 4ms/step - loss: 0.0724 - acc: 0.9765 - val_loss: 0.1189 - val_acc: 0.9651
Epoch 6/10
18000/18000 [==============================] - 64s 4ms/step - loss: 0.0542 - acc: 0.9828 - val_loss: 0.1254 - val_acc: 0.9641
Epoch 7/10
18000/18000 [==============================] - 69s 4ms/step - loss: 0.0420 - acc: 0.9869 - val_loss: 0.1377 - val_acc:

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi